In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614win.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [52]:
f_dir1 = 'D:/OneDrive/Develop/Smart_Fishery/models/ICROS/Train_0/model.ckpt-2280.meta'
f_dir2 = '/models/ICROS/Train_0/'

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_label.shape[0]/batch)
ev_acc = np.zeros(ev_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob = graph.get_tensor_by_name("dropout/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_data[batch*i:batch*(i+1)], keep_prob:1.0}
            label = ev_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_data[batch*i:], keep_prob:1.0}
            temp = ev_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/ICROS/Train_5/model.ckpt-2280
[[4.1104868e-04 9.9958891e-01]
 [9.4017677e-04 9.9905986e-01]
 [1.0000000e+00 1.5728530e-17]
 [1.0000000e+00 1.9774897e-10]
 [1.0000000e+00 3.9428169e-10]
 [1.0801660e-04 9.9989200e-01]
 [5.2425492e-04 9.9947578e-01]
 [1.2300725e-04 9.9987698e-01]
 [1.0000000e+00 3.7959345e-12]
 [1.7425407e-03 9.9825746e-01]
 [5.4375008e-03 9.9456257e-01]
 [9.9997342e-01 2.6527703e-05]
 [2.5159344e-10 1.0000000e+00]
 [2.8694133e-04 9.9971300e-01]
 [7.9865385e-05 9.9992013e-01]
 [1.0000000e+00 2.2853095e-12]
 [3.9904808e-06 9.9999607e-01]
 [1.3168876e-10 1.0000000e+00]
 [8.9465594e-01 1.0534403e-01]
 [8.9253961e-05 9.9991071e-01]
 [9.9131948e-01 8.6804731e-03]
 [4.1710604e-02 9.5828944e-01]
 [9.9988687e-01 1.1305915e-04]
 [9.9788982e-01 2.1101334e-03]
 [9.4495487e-01 5.5045158e-02]
 [9.9999988e-01 7.4018054e-08]
 [9.1489939e-19 1.0000000e+00]
 [3.2502241e-03 9.9674976e-01]
 [1.0000000e+00 2.0542226e-09]
 [3.8610136e-03 9.961

In [53]:
print(ev_acc[-1])
print(np.mean(ev_acc))

1.0
0.9638949671772429


In [54]:
final = np.array([0.9398, 0.9517, 0.9639, 0.9737, 0.9814, 0.9639, 0.9595, 0.8742, 0.9814, 0.9825])
print(np.mean(final))
print(np.std(final))

0.9572
0.03066056098638773


In [55]:
fail_idx = np.where(ev_acc == 0)
print(fail_idx[0])

[ 18  46  63  83  85 104 196 224 302 310 376 400 408 424 439 486 503 510
 555 563 574 626 654 665 670 674 708 748 752 760 797 822 852]


In [47]:
"""
for i in range(len(fail_idx[0])):
    new_img = cv2.imread(tr_data_dir['test'][fail_idx[0][i]][0])
    
    save_path = "/models/ICROS-fail/T9_fail-%d.png" % i
    cv2.imwrite(save_path, new_img)
"""